In [1]:
import tensorflow as tf
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow_transform as tft
import tempfile
import tensorflow_transform.beam.impl as tft_beam
import apache_beam as beam
import pprint

Error importing tfx_bsl_extension.arrow.array_util. Some tfx_bsl functionalities are not available

In [5]:
raw_data = [
    {'x':   1.20},
    {'x':   2.99},
    {'x': 100.00}
]

tfrecords_filename = "transformexample.tfrecords"
tf_record_writer = tf.io.TFRecordWriter(tfrecords_filename)

for row in raw_data[:]:    
    record_bytes = tf.train.Example(
        features=tf.train.Features(
            feature={
                'x': tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=[row['x']]
                    )
                )
            }
        )
    ).SerializeToString()
    tf_record_writer.write(record_bytes)
tf_record_writer.close()

In [6]:
def preprocessing_fn(inputs):
    x = inputs['x']
    x_normalized = tft.scale_to_0_1(x)
    return {
        'x_xf': x_normalized
    }

In [8]:
raw_data_metadata = dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec({
        'x': tf.io.FixedLenFeature([], tf.float32),
    }))
data_coder = tft.coders.ExampleProtoCoder(raw_data_metadata.schema)

with beam.Pipeline() as pipeline:
    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
        raw_data = (
            pipeline | beam.io.ReadFromTFRecord(tfrecords_filename, coder=data_coder))
        
        (transformed_data, transformed_metadata), transform_fn = (  
            (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(
                preprocessing_fn))
            
        transformed_data | beam.Map(print)

#print('\nRaw data:\n{}\n'.format(pprint.pformat(raw_data)))
#print('Transformed data:\n{}'.format(pprint.pformat(transformed_data)))

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


{'x_xf': 0.0}
{'x_xf': 0.018117407}
{'x_xf': 1.0}

Raw data:
<PCollection[[8]: ReadFromTFRecord/Read/_SDFBoundedSourceWrapper/ParDo(SDFBoundedSourceDoFn).None] at 0x7f9b1bffc190>

Transformed data:
<PCollection[[8]: AnalyzeAndTransformDataset/TransformDataset/ConvertAndUnbatch.None] at 0x7f9b1b76f4d0>
